FloPy
MNW2 package example
Link to original flopy jupnb tutorial: `MNW2_tutorial <https://github.com/modflowpy/flopy/blob/develop/examples/Notebooks/flopy3_mnw2package_example.ipynb>`_ 

In [1]:
import sys
import os
from tempfile import TemporaryDirectory

import numpy as np

try:
    import pandas as pd
except:
    pass

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join("..", ".."))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print("numpy version: {}".format(np.__version__))
try:
    print("pandas version: {}".format(pd.__version__))
except:
    pass
print("flopy version: {}".format(flopy.__version__))

3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.22.0
pandas version: 1.3.5
flopy version: 3.3.5


Make an MNW2 package from scratch

In [2]:
# temporary directory
temp_dir = TemporaryDirectory()
model_ws = temp_dir.name

m = flopy.modflow.Modflow("mnw2example", model_ws=model_ws)
dis = flopy.modflow.ModflowDis(
    nrow=5, ncol=5, nlay=3, nper=3, top=10, botm=0, model=m
)

MNW2 information by node
(this could be prepared externally from well records and read in from a csv or excel file)

this table has two multi-node wells, the first (well1) consisting of two nodes that are manually specified (where the variable rw is specified by node)
node that some variables that are constant for the whole well are also included (losstype, zpump, etc.)

In [3]:
node_data = pd.DataFrame(
    [
        [1, 1, 9.5, 7.1, "well1", "skin", -1, 0, 0, 0, 1.0, 2.0, 5.0, 6.2],
        [1, 1, 7.1, 5.1, "well1", "skin", -1, 0, 0, 0, 0.5, 2.0, 5.0, 6.2],
        [3, 3, 9.1, 3.7, "well2", "skin", -1, 0, 0, 0, 1.0, 2.0, 5.0, 4.1],
    ],
    columns=[
        "i",
        "j",
        "ztop",
        "zbotm",
        "wellid",
        "losstype",
        "pumploc",
        "qlimit",
        "ppflag",
        "pumpcap",
        "rw",
        "rskin",
        "kskin",
        "zpump",
    ],
)
node_data

,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,pumpcap,rw,rskin,kskin,zpump
0,1,1,9.5,7.1,well1,skin,-1,0,0,0,1.0,2.0,5.0,6.2
1,1,1,7.1,5.1,well1,skin,-1,0,0,0,0.5,2.0,5.0,6.2
2,3,3,9.1,3.7,well2,skin,-1,0,0,0,1.0,2.0,5.0,4.1


convert the DataFrame to a recarray for compatibility with flopy

In [4]:
node_data = node_data.to_records()
node_data

rec.array([(0, 1, 1, 9.5, 7.1, 'well1', 'skin', -1, 0, 0, 0, 1. , 2., 5., 6.2),
           (1, 1, 1, 7.1, 5.1, 'well1', 'skin', -1, 0, 0, 0, 0.5, 2., 5., 6.2),
           (2, 3, 3, 9.1, 3.7, 'well2', 'skin', -1, 0, 0, 0, 1. , 2., 5., 4.1)],
          dtype=[('index', '<i8'), ('i', '<i8'), ('j', '<i8'), ('ztop', '<f8'), ('zbotm', '<f8'), ('wellid', 'O'), ('losstype', 'O'), ('pumploc', '<i8'), ('qlimit', '<i8'), ('ppflag', '<i8'), ('pumpcap', '<i8'), ('rw', '<f8'), ('rskin', '<f8'), ('kskin', '<f8'), ('zpump', '<f8')])

Stress period information
(could also be developed externally)

In [6]:
stress_period_data = pd.DataFrame(
    [
        [0, "well1", 0],
        [1, "well1", 100.0],
        [0, "well2", 0],
        [1, "well2", 1000.0],
    ],
    columns=["per", "wellid", "qdes"],
)
stress_period_data

,per,wellid,qdes
0,0,well1,0.0
1,1,well1,100.0
2,0,well2,0.0
3,1,well2,1000.0


In [7]:
pers = stress_period_data.groupby("per")
stress_period_data = {i: pers.get_group(i).to_records() for i in [0, 1]}
stress_period_data

{0: rec.array([(0, 0, 'well1', 0.), (2, 0, 'well2', 0.)],
           dtype=[('index', '<i8'), ('per', '<i8'), ('wellid', 'O'), ('qdes', '<f8')]),
 1: rec.array([(1, 1, 'well1',  100.), (3, 1, 'well2', 1000.)],
           dtype=[('index', '<i8'), ('per', '<i8'), ('wellid', 'O'), ('qdes', '<f8')])}

Make ModflowMnw2 package object
note that extraneous columns in node_data and stress_period_data are ignored
if itmp is positive, it must equal the number of active wells being specified in stress_period_data, otherwise the package class will raise an error.

In [8]:
mnw2 = flopy.modflow.ModflowMnw2(
    model=m,
    mnwmax=2,
    node_data=node_data,
    stress_period_data=stress_period_data,
    itmp=[2, 2, -1],  # reuse second per pumping for last stress period
)

In [9]:
# "nodtot" is computed automatically
mnw2.nodtot

3

In [10]:
pd.DataFrame(mnw2.node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,3,3,9.1,3.7,well2,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pd.DataFrame(mnw2.stress_period_data[0])

,k,i,j,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,1,1,well1,0.0,0,0.0,0.0,0,0.0,0.0
1,0,3,3,well2,0.0,0,0.0,0.0,0,0.0,0.0


In [12]:
pd.DataFrame(mnw2.stress_period_data[1])

,k,i,j,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,1,1,well1,100.0,0,0.0,0.0,0,0.0,0.0
1,0,3,3,well2,1000.0,0,0.0,0.0,0,0.0,0.0


In [13]:
tmp = flopy.modflow.ModflowMnw2(
    model=m,
    itmp=[1, 1, -1],  # reuse second per pumping for last stress period
)

empty node_data and stress_period_data tables can also be generated by the package class, and then filled

In [14]:
node_data = tmp.get_empty_node_data(3)
node_data

rec.array([(0, 0, 0, 0., 0., 0, 0, 0, 0, 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0, 0, 0, 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0.),
           (0, 0, 0, 0., 0., 0, 0, 0, 0, 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0, 0, 0, 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0.),
           (0, 0, 0, 0., 0., 0, 0, 0, 0, 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0, 0, 0, 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0.)],
          dtype=[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('ztop', '<f4'), ('zbotm', '<f4'), ('wellid', 'O'), ('losstype', 'O'), ('pumploc', '<i4'), ('qlimit', '<i4'), ('ppflag', '<i4'), ('pumpcap', '<i4'), ('rw', '<f4'), ('rskin', '<f4'), ('kskin', '<f4'), ('B', '<f4'), ('C', '<f4'), ('P', '<f4'), ('cwc', '<f4'), ('pp', '<f4'), ('pumplay', '<i4'), ('pumprow', '<i4'), ('pumpcol', '<i4'), ('zpump', '<f4'), ('hlim', '<f4'), ('qcut', '<i4'), ('qfrcmn', '<f4'), ('qfrcmx', '<f4'), ('hlift', '<f4'), ('liftq0', '<f4'), ('liftqmax', '<f4'), ('hwtol', '<f4'), ('liftn', '<f4'), ('qn', '<f4')])

Mnw objects
at the base of the flopy mnw2 module is the Mnw object class, which describes a single multi-node well. A list or dict of Mnw objects can be used to build a package (using the example above):

    flopy.modflow.ModflowMnw2(model=m, mnwmax=2,
                 mnw=<dict or list of Mnw objects>,
                 itmp=[1, 1, -1], # reuse second per pumping for last stress period
                 )
or if node_data and stress_period_data are supplied, the Mnw objects are created on initialization of the ModflowMnw2 class instance, and assigned to the .mnw attribute, as items in a dictionary keyed by wellid.

In [15]:
mnw2.mnw

{'well1': <flopy.modflow.mfmnw2.Mnw at 0x2001097f910>,
 'well2': <flopy.modflow.mfmnw2.Mnw at 0x2001097f8b0>}

In [16]:
mnw2.mnw["well1"].__dict__

{'nper': 3,
 'mnwpackage': 
     Multi-Node Well 2 Package Class
 
     Parameters
     ----------
     model : model object
         The model object (of type :class:'flopy.modflow.mf.Modflow') to which
         this package will be added.
     mnwmax : int
         The absolute value of MNWMAX is the maximum number of multi-node wells
         (MNW) to be simulated. If MNWMAX is a negative number, NODTOT is read.
     nodtot : int
         Maximum number of nodes.
         The code automatically estimates the maximum number of nodes (NODTOT)
         as required for allocation of arrays. However, if a large number of
         horizontal wells are being simulated, or possibly for other reasons,
         this default estimate proves to be inadequate, a new input option has
         been added to allow the user to directly specify a value for NODTOT.
         If this is a desired option, then it can be implemented by specifying
         a negative value for "MNWMAX"--the first value lis

{'nper': 3,
 'mnwpackage': 
     Multi-Node Well 2 Package Class
 
     Parameters
     ----------
     model : model object
         The model object (of type :class:'flopy.modflow.mf.Modflow') to which
         this package will be added.
     mnwmax : int
         The absolute value of MNWMAX is the maximum number of multi-node wells
         (MNW) to be simulated. If MNWMAX is a negative number, NODTOT is read.
     nodtot : int
         Maximum number of nodes.
         The code automatically estimates the maximum number of nodes (NODTOT)
         as required for allocation of arrays. However, if a large number of
         horizontal wells are being simulated, or possibly for other reasons,
         this default estimate proves to be inadequate, a new input option has
         been added to allow the user to directly specify a value for NODTOT.
         If this is a desired option, then it can be implemented by specifying
         a negative value for "MNWMAX"--the first value listed in Record 1
         (Line 1) of the MNW2 input data file. If this is done, then the code
         will assume that the very next value on that line will be the desired
         value of "NODTOT". The model will then reset "MNWMAX" to its absolute
         value. The value of "ipakcb" will become the third value on that
         line, etc.
     ipakcb : int
         is a flag and a unit number:
             if ipakcb > 0, then it is the unit number to which MNW cell-by-cell
             flow terms will be recorded whenever cell-by-cell budget data are
             written to a file (as determined by the outputcontrol options of
             MODFLOW).
             if ipakcb = 0, then MNW cell-by-cell flow terms will not be printed
                 or recorded.
             if ipakcb < 0, then well injection or withdrawal rates and water
                 levels in the well and its multiple cells will be printed in
                 the main MODFLOW listing (output) file whenever cell-by-cell
                 budget data are written to a file (as determined by the output
                 control options of MODFLOW).
     mnwprnt : integer
         Flag controlling the level of detail of information about multi-node
         wells to be written to the main MODFLOW listing (output) file.
         If MNWPRNT = 0, then only basic well information will be printed in
         the main MODFLOW output file; increasing the value of MNWPRNT yields
         more information, up to a maximum level of detail corresponding
         with MNWPRNT = 2. (default is 0)
     aux : list of strings
         (listed as "OPTION" in MNW2 input instructions)
         is an optional list of character values in the style of "AUXILIARY abc"
         or "AUX abc" where "abc" is the name of an auxiliary parameter to be
         read for each multi-node well as part of dataset 4a. Up to 20
         parameters can be specified, each of which must be preceded by
         "AUXILIARY" or "AUX." These parameters will not be used by the MNW2
         Package, but they will be available for use by other packages.
         (default is None)
     node_data : numpy record array
         master table describing multi-node wells in package. Same format as
         node_data tables for each Mnw object. See Mnw class documentation for
         more information.
     mnw : list or dict of Mnw objects
         Can be supplied instead of node_data and stress_period_data tables
         (in which case the tables are constructed from the Mnw objects).
         Otherwise the a dict of Mnw objects (keyed by wellid) is constructed
         from the tables.
     stress_period_data : dict of numpy record arrays
         master dictionary of record arrays (keyed by stress period) containing
         transient input for multi-node wells. Format is the same as stress
         period data for individual Mnw objects, except the 'per' column is
         replaced by 'wellid' (containing wellid for each MNW). See Mnw class
         documentation for more information.
     itmp : list of ints
         is an integer value for reusing or reading multi-node well data; it
         can change each stress period. ITMP must be >= 0 for the first stress
         period of a simulation.
         if ITMP > 0, then ITMP is the total number of active multi-node wells
             simulated during the stress period, and only wells listed in
             dataset 4a will be active during the stress period. Characteristics
             of each well are defined in datasets 2 and 4.
         if ITMP = 0, then no multi-node wells are active for the stress period
             and the following dataset is skipped.
         if ITMP < 0, then the same number of wells and well information will
             be reused from the previous stress period and dataset 4 is skipped.
     extension : string
         Filename extension (default is 'mnw2')
     unitnumber : int
         File unit number (default is None).
     filenames : str or list of str
         Filenames to use for the package and the output files. If
         filenames=None the package name will be created using the model name
         and package extension and the cbc output name will be created using
         the model name and .cbc extension (for example, modflowtest.cbc),
         if ipakcbc is a number greater than zero. If a single string is passed
         the package will be set to the string and cbc output names will be
         created using the model name and .cbc extension, if ipakcbc is a
         number greater than zero. To define the names for all package files
         (input and output) the length of the list of strings should be 2.
         Default is None.
     gwt : boolean
         Flag indicating whether GW transport process is active
 
     Attributes
     ----------
 
     Methods
     -------
 
     See Also
     --------
 
     Notes
     -----
 
     Examples
     --------
 
     >>> import flopy
     >>> ml = flopy.modflow.Modflow()
     >>> mnw2 = flopy.modflow.ModflowMnw2(ml, ...)
 
      _name = MNW2
  _parent = MODFLOW 3 layer(s) 5 row(s) 5 column(s) 3 stress period(s) ('flopy.modflow.mf.Modflow)
  acceptable_dtypes (list, items = 3)
  allowDuplicates = False ('bool)
  aux (list, items = 0)
  file_name = mnw2example.mnw2
  fn_path = /var/folders/4t/2sbpjbgs1n73t8s7rx4gb_ph0000gn/T/tmpp2621gci/mnw2example.mnw2 ('str)
  gwt = False ('bool)
  ipakcb = 0 ('int)
  itmp (list, items = 3)
  mnw = {'well1': <flopy.modflow.mfmnw2.Mnw object at 0x11d215ed0>, 'well2': <flopy.modflow.mfmnw2.Mnw object at 0x108c372e0>} ('dict)
  mnwmax = 2 ('int)
  mnwprnt = 0 ('int)
  node_data (array, shape = 3,)
  nodtot = 3 ('int)
  nper = 3 ('int)
  stress_period_data = <flopy.utils.util_list.MfList object at 0x11d215f90> ('flopy.utils.util_list.MfList)
  structured = True ('bool)
  unit_number = 34,
 'aux': [],
 'wellid': 'well1',
 'nnodes': -2,
 'losstype': 'skin',
 'pumploc': -1,
 'qlimit': 0,
 'ppflag': 0,
 'pumpcap': 0,
 'rw': [1.0, 0.5],
 'rskin': [2.0, 2.0],
 'kskin': [5.0, 5.0],
 'B': [None],
 'C': [0],
 'P': [2.0],
 'cwc': [None],
 'pp': [1],
 'k': [0],
 'i': [1, 1],
 'j': [1, 1],
 'ztop': [9.5, 7.1],
 'zbotm': [7.1, 5.1],
 'pumplay': 0,
 'pumprow': 0,
 'pumpcol': 0,
 'zpump': 6.2,
 'hlim': None,
 'qcut': None,
 'qfrcmn': None,
 'qfrcmx': None,
 'hlift': None,
 'liftq0': None,
 'liftqmax': None,
 'hwtol': None,
 'liftn': None,
 'qn': None,
 'stress_period_data': rec.array([(0, 1, 1, 0,   0., 0, 0., 0., 0, 0., 0.),
            (0, 1, 1, 1, 100., 0, 0., 0., 0, 0., 0.),
            (0, 1, 1, 1, 100., 0, 0., 0., 0, 0., 0.)],
           dtype=[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('per', '<i8'), ('qdes', '<f4'), ('capmult', '<i8'), ('cprime', '<f4'), ('hlim', '<f4'), ('qcut', '<i8'), ('qfrcmn', '<f4'), ('qfrcmx', '<f4')]),
 'node_data': rec.array([(0, 1, 1, 9.5, 7.1, 'well1', 'skin', -1, 0, 0, 0, 1. , 2., 5., 0., 0., 0., 0., 0., 0, 0, 0, 6.2, 0., 0, 0., 0., 0., 0., 0., 0., 0., 0.),
            (0, 1, 1, 7.1, 5.1, 'well1', 'skin', -1, 0, 0, 0, 0.5, 2., 5., 0., 0., 0., 0., 0., 0, 0, 0, 6.2, 0., 0, 0., 0., 0., 0., 0., 0., 0., 0.)],
           dtype=[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('ztop', '<f4'), ('zbotm', '<f4'), ('wellid', 'O'), ('losstype', 'O'), ('pumploc', '<i8'), ('qlimit', '<i8'), ('ppflag', '<i8'), ('pumpcap', '<i8'), ('rw', '<f4'), ('rskin', '<f4'), ('kskin', '<f4'), ('B', '<f4'), ('C', '<f4'), ('P', '<f4'), ('cwc', '<f4'), ('pp', '<f4'), ('pumplay', '<i8'), ('pumprow', '<i8'), ('pumpcol', '<i8'), ('zpump', '<f4'), ('hlim', '<f4'), ('qcut', '<i8'), ('qfrcmn', '<f4'), ('qfrcmx', '<f4'), ('hlift', '<f4'), ('liftq0', '<f4'), ('liftqmax', '<f4'), ('hwtol', '<f4'), ('liftn', '<f4'), ('qn', '<f4')])}
Note that Mnw object attributes for variables that vary by node are lists (e.g. rw above)

Each Mnw object has its own node_data and stress_period_data

In [17]:
pd.DataFrame(mnw2.mnw["well1"].node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Instead of a dict keyed by stress period, Mnw.stress_period_data is a recarray with pumping data listed by stress period for that well
note that data for period 2, where itmp < 1, is shown (was copied from s.p. 1 during construction of the Mnw object)

In [18]:

pd.DataFrame(mnw2.mnw["well2"].stress_period_data)

,k,i,j,per,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,3,3,0,0.0,0,0.0,0.0,0,0.0,0.0
1,0,3,3,1,1000.0,0,0.0,0.0,0,0.0,0.0
2,0,3,3,1,1000.0,0,0.0,0.0,0,0.0,0.0


Build the same package using only the Mnw objects

In [19]:
mnw2fromobj = flopy.modflow.ModflowMnw2(
    model=m,
    mnwmax=2,
    mnw=mnw2.mnw,
    itmp=[2, 2, -1],  # reuse second per pumping for last stress period
)

In [20]:
pd.DataFrame(mnw2fromobj.node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,3,3,9.1,3.7,well2,skin,-1,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
pd.DataFrame(mnw2fromobj.stress_period_data[0])

,k,i,j,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,1,1,well1,0.0,0,0.0,0.0,0,0.0,0.0
1,0,3,3,well2,0.0,0,0.0,0.0,0,0.0,0.0


In [22]:
pd.DataFrame(mnw2fromobj.stress_period_data[1])

,k,i,j,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,1,1,well1,100.0,0,0.0,0.0,0,0.0,0.0
1,0,3,3,well2,1000.0,0,0.0,0.0,0,0.0,0.0


By default, the node_data and stress_period_data tables attached to the ModflowMnw2 package class are definitive
* on writing of the package output (mnw2.write_file()), the Mnw objects are regenerated from the tables. This setting is controlled by the default argument use_tables=True. To write the package file using the Mnw objects (ignoring the tables), use mnw2.write_file(use_tables=False).

In [23]:
per1 = flopy.modflow.ModflowMnw2.get_empty_stress_period_data(itmp=2)
per1

rec.array([(0, 0, 0, 0, 0., 0, 0., 0., 0, 0., 0.),
           (0, 0, 0, 0, 0., 0, 0., 0., 0, 0., 0.)],
          dtype=[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('wellid', 'O'), ('qdes', '<f4'), ('capmult', '<i4'), ('cprime', '<f4'), ('hlim', '<f4'), ('qcut', '<i4'), ('qfrcmn', '<f4'), ('qfrcmx', '<f4')])

Write an MNW2 package file and inspect the results

In [24]:
mnw2.write_file(os.path.join(model_ws, "test.mnw2"))

In [25]:
junk = [print(l.strip("\n")) for l in open(os.path.join(model_ws, "test.mnw2")).readlines()]

# MNW2 package for MODFLOW-2005 generated by Flopy 3.3.5
2 0 0
well1 -2
            skin -1 0 0 0
              -1.0000000E+00    2.0000000E+00    5.0000000E+00
               9.5000000E+00    7.0999999E+00 2 2    1.0000000E+00
               7.0999999E+00    5.0999999E+00 2 2    5.0000000E-01
               6.1999998E+00
well2 -1
            skin -1 0 0 0
               1.0000000E+00    2.0000000E+00    5.0000000E+00
               9.1000004E+00    3.7000000E+00 4 4
               4.0999999E+00
2  Stress Period 1
well1    0.0000000E+00
well2    0.0000000E+00
2  Stress Period 2
well1    1.0000000E+02
well2    1.0000000E+03
-1  Stress Period 3
